In [1]:
import pandas as pd
from pathlib import PosixPath
import ast

def clean_path_list(path_str):
    try:
        # Reemplazar PosixPath(...) por comillas
        path_str = path_str.replace("PosixPath(", "").replace(")", "")
        evaluated = ast.literal_eval(path_str)
        return [str(p) for p in evaluated]
    except Exception as e:
        print(f"Error al parsear: {path_str}\n{e}")
        return []


In [2]:
import os
import torch
import torchaudio
import pandas as pd
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel, Wav2Vec2Model, Wav2Vec2Processor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tqdm import tqdm
import ast
from pathlib import Path
import gc

import torch
import torchaudio
from torch.utils.data import Dataset

class MultimodalFallacyDataset(Dataset):
    def __init__(self, df, tokenizer, processor, audio_dir, label_encoder=None, max_length=128):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.processor = processor
        self.audio_dir = audio_dir
        self.max_length = max_length
        self.label_encoder = label_encoder

        if label_encoder:
            self.df['label'] = label_encoder.transform(self.df['Etiqueta'])

        self.min_audio_len =  1600  # 100ms at 16kHz, padding threshold

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Text input
        text = row['Texto']
        inputs = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")

        try:
            audio_paths = row['Ruta']
            if isinstance(audio_paths, str):
                audio_paths = ast.literal_eval(audio_paths)
            if not isinstance(audio_paths, list):
                audio_paths = [audio_paths]

            waveforms = []
            for path in audio_paths:
                try:
                    waveform, sr = torchaudio.load(path)

                    if sr != 16000:
                        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
                        waveform = resampler(waveform)

                    waveform = waveform.mean(dim=0, keepdim=True)

                    if waveform.numel() == 0:
                        raise ValueError(f"Audio vacío: {path}")

                    waveforms.append(waveform)

                except Exception as e:
                    print(f"Error al cargar el audio {path}: {e}. Se usará tensor de ceros.")
                    waveforms.append(torch.zeros(1, self.min_audio_len))

            waveform = torch.cat(waveforms, dim=1)

        except Exception as e:
            print(f"Error procesando 'Ruta' para índice {idx}: {row['Ruta']} - Error: {str(e)}")
            waveform = torch.zeros(1, self.min_audio_len)

        if waveform.shape[1] < self.min_audio_len:
            pad_size = self.min_audio_len - waveform.shape[1]
            waveform = torch.nn.functional.pad(waveform, (0, pad_size), value=0)

        # Imprimir la ruta y la longitud antes de procesar
        print(f"[DEBUG] Procesando audio del índice {idx}, ruta(s): {row['Ruta']} - longitud tras concat: {waveform.shape[1]}")

        audio_input = self.processor(
            waveform.squeeze(),
            sampling_rate=16000,
            return_tensors="pt",
            padding=True,
            truncation=False,
            return_attention_mask=True,
            mask_time_prob=0.0,
            mask_time_length=0
        )

        input_len = audio_input['input_values'].shape[-1]
        if input_len < 10:
            raise ValueError(f"Audio demasiado corto tras procesar: {row['Ruta']} -> longitud: {input_len}")

        item = {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'input_values': audio_input['input_values'].squeeze(0),
        }

        if 'label' in row:
            item['label'] = torch.tensor(row['label'], dtype=torch.long)

        return item




class MultimodalClassifier(nn.Module):
    def __init__(self, text_model_name='roberta-base', audio_model_name='facebook/wav2vec2-base', num_labels=6):
        super().__init__()
        self.text_encoder = RobertaModel.from_pretrained(text_model_name)
        self.audio_encoder = Wav2Vec2Model.from_pretrained(audio_model_name)

        # Freeze audio encoder
        self.audio_encoder.eval()
        for param in self.audio_encoder.parameters():
            param.requires_grad = False

        text_hidden = self.text_encoder.config.hidden_size
        audio_hidden = self.audio_encoder.config.hidden_size

        self.classifier = nn.Sequential(
            nn.Linear(text_hidden + audio_hidden, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_labels)
        )

    def forward(self, input_ids, attention_mask, input_values):
        text_output = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        audio_output = self.audio_encoder(input_values=input_values)

        text_feat = text_output.last_hidden_state[:, 0]  # [CLS]
        audio_feat = audio_output.last_hidden_state.mean(dim=1)

        fused = torch.cat((text_feat, audio_feat), dim=1)
        return self.classifier(fused)

from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()

import gc

def train_model(model, dataloader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    
    for batch in tqdm(dataloader):
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        input_values = batch['input_values'].to(device)
        labels = batch['label'].to(device)

        try:
            with autocast():
                outputs = model(input_ids, attention_mask, input_values)
                loss = loss_fn(outputs, labels)

        except Exception as e:
            raise e  # para seguir viendo el traceback

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

        del input_ids, attention_mask, input_values, labels, outputs, loss
        torch.cuda.empty_cache()
        gc.collect()

    return total_loss / len(dataloader)

from sklearn.metrics import classification_report, f1_score, accuracy_score

def evaluate_model(model, dataloader, device, label_encoder):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            input_values = batch['input_values'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask, input_values)
            pred = torch.argmax(outputs, dim=1)

            preds.extend(pred.cpu().numpy())
            targets.extend(labels.cpu().numpy())

            # Liberar memoria
            del input_ids, attention_mask, input_values, labels, outputs, pred
            torch.cuda.empty_cache()
            import gc; gc.collect()

    # Métricas
    accuracy = accuracy_score(targets, preds)
    f1_weighted = f1_score(targets, preds, average='weighted', zero_division=0)
    f1_macro = f1_score(targets, preds, average='macro', zero_division=0)

    print("Evaluation Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Weighted: {f1_weighted:.4f}")
    print(f"F1 Macro: {f1_macro:.4f}")

    print("\nDetailed Classification Report:")
    print(classification_report(targets, preds, target_names=label_encoder.classes_, zero_division=0))

/home/evacantin/miniconda3/envs/haia/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_55218/975103250.py:145: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [3]:
def custom_collate_fn(batch):
    # Filtra los ejemplos donde el input_values es más corto que mask_length
    mask_length = 10  # Esto depende de tu configuración de Wav2Vec2
    filtered_batch = [item for item in batch if item['input_values'].shape[-1] >= mask_length]

    if len(filtered_batch) == 0:
        raise ValueError("Todos los ejemplos del batch son demasiado cortos para mask_length.")

    input_ids = torch.stack([item['input_ids'] for item in filtered_batch])
    attention_mask = torch.stack([item['attention_mask'] for item in filtered_batch])
    
    input_values = [item['input_values'] for item in filtered_batch]
    max_len = max([x.shape[-1] for x in input_values])
    padded_values = torch.stack([torch.nn.functional.pad(x, (0, max_len - x.shape[-1])) for x in input_values])

    labels = torch.stack([item['label'] for item in filtered_batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'input_values': padded_values,
        'label': labels
    }


In [4]:
import ast
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_df = pd.read_csv("/mnt/c/Users/EvaCantin/Desktop/MASTER/HAIA/data_text_audio/MMUSED-fallacy/afc_text_audio_train.csv")
    test_df = pd.read_csv("/mnt/c/Users/EvaCantin/Desktop/MASTER/HAIA/data_text_audio/MMUSED-fallacy/afc_text_audio_test.csv")
    train_df['Ruta'] = train_df['Ruta'].apply(clean_path_list)
    test_df['Ruta'] = test_df['Ruta'].apply(clean_path_list)
    
    train_df['Ruta'] = train_df['Ruta'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    mask = ~train_df['Ruta'].apply(lambda paths: any("36_2008/933" in path or "36_2008/934" in path or "6_1976/543" in path or "36_2008/44" in path or "30_2004/456" in path or "30_2004/230" in path or "33_2008/949" in path for path in paths))
    train_df = train_df[mask].reset_index(drop=True)
    label_encoder = LabelEncoder().fit(train_df['Etiqueta'])

    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base')
    
    #processor.feature_extractor.mask_time_length = 2
    train_dataset = MultimodalFallacyDataset(train_df, tokenizer, processor, "/mnt/c/Users/EvaCantin/Desktop/MASTER/HAIA/Trabajo/MMUSED-fallacy/audio_clips", label_encoder)
    test_dataset = MultimodalFallacyDataset(test_df.dropna(subset=['Etiqueta']), tokenizer, processor, "/mnt/c/Users/EvaCantin/Desktop/MASTER/HAIA/Trabajo/MMUSED-fallacy/audio_clips", label_encoder)

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=custom_collate_fn, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=1, collate_fn=custom_collate_fn, num_workers=2)

    model = MultimodalClassifier().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(3):
        print(f"Epoch {epoch+1}")
        train_loss = train_model(model, train_loader, optimizer, loss_fn, device)
        print(f"Train loss: {train_loss:.4f}")
        evaluate_model(model, test_loader, device, label_encoder)
        torch.cuda.empty_cache()

/home/evacantin/miniconda3/envs/haia/lib/python3.10/site-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/evacantin/miniconda3/envs/haia/lib/python3.10/site-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Tr

Epoch 1


  0%|          | 0/1220 [00:00<?, ?it/s]

[DEBUG] Procesando audio del índice 56, ruta(s): ['/mnt/c/users/evacantin/desktop/MASTER/HAIA/Trabajo/MMUSED-fallacy/audio_clips/18_1992/233.wav', '/mnt/c/users/evacantin/desktop/MASTER/HAIA/Trabajo/MMUSED-fallacy/audio_clips/18_1992/234.wav'] - longitud tras concat: 28832


Keyword argument `mask_time_prob` is not a valid argument for this processor and will be ignored.
Keyword argument `mask_time_length` is not a valid argument for this processor and will be ignored.


...

[DEBUG] Procesando audio del índice 84, ruta(s): ['/mnt/c/users/evacantin/desktop/MASTER/HAIA/Trabajo/MMUSED-fallacy/audio_clips/1_1960/22.wav'] - longitud tras concat: 40336


 56%|█████▌    | 681/1220 [03:05<02:18,  3.90it/s]

[DEBUG] Procesando audio del índice 1126, ruta(s): ['/mnt/c/users/evacantin/desktop/MASTER/HAIA/Trabajo/MMUSED-fallacy/audio_clips/45_2020/423.wav'] - longitud tras concat: 70784


 56%|█████▌    | 681/1220 [03:05<02:26,  3.68it/s]


ValueError: `mask_length` has to be smaller than `sequence_length`, but got `mask_length`: 10 and `sequence_length`: 8`